#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 05 - Containers 2 - Exercises</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

### 02 Exercises

This exercise will cover GPU computation with docker images and Docker Hub.

### Installation

We need to install the Nvidia Container Toolkit, instructions are [here](https://github.com/NVIDIA/nvidia-docker).

*NB: At some point in the past on Ubuntu 16.04 I had to follow [these instructions](https://cnvrg.io/how-to-setup-docker-and-nvidia-docker-2-0-on-ubuntu-18-04) for getting the correct version of Docker and the Nvidia runtime. This was with a version of Docker before 19.03, so check your version with `docker version` to make sure you have at least that version otherwise those instructions might be necessary.*


### Exercise 1:

We'll write a simple Pytorch program to produce some information about the environment they're being run under to see what a CUDA-aware container looks like.

**Step 1:** Create a directory called `exercise02` and copy the following into it as `pytorch_test.py`:

In [ ]:
import os
import torch
import timeit

print(f'Host name: {os.environ["HOSTNAME"]}')
print(f'Pytorch version: {torch.__version__}')

for d in range(torch.cuda.device_count()):
    p = torch.cuda.get_device_properties(d)
    print(f"Device {d} is {p.name} with {p.total_memory/2**30}GiB of memory")
    
test=torch.rand(10,1,64,64)
conv=torch.nn.Conv2d(1,1,3,1,1)

result=timeit.timeit("conv(test)",number=1000,globals=locals())
print(f'CPU time: {result}')

test=test.to('cuda:0')
conv=conv.to('cuda:0')

result=timeit.timeit("conv(test)",number=1000,globals=locals())
print(f'GPU time: {result}')

**Step 2:** Now define the Dockerfile based off one of the CUDA [images of your choice](https://hub.docker.com/r/nvidia/cuda/tags). You'll need to install Pytorch so refer to the notes for how to do that. The Dockerfile should copy over the test file we just created and run it as the command.

**Step 3:** build the image tagging it as `pytorch-test`:

**Step 4:** Run the image with and without the GPU flag (refer to the notes) to see how it works:

**Step 5:** So far we've been using `CMD` to define the command to run. A related Dockerfile command is `ENTRYPOINT` which provides a command to run but is not replaced when arguments are used with `docker run`. With `CMD` any extra arguments will replace what was provided, but with `ENTRYPOINT` they are appended to the end of the command to act as additional arguments. Both can be used at once, `ENTRYPOINT` defining the command and `CMD` giving default arguments.

Let's modify our Dockerfile to run the given script as the default behaviour but allow any other arguments to be used as well. We can define a general purpose container this way which can used to execute any script we provided through a volume directory.

### Exercise 2:
Make an account on Docker Hub and upload your new image.

### Exercise 3:

Docker Swarm allows us to run instances of Docker images on worker nodes commanded from a manager node. Typically these nodes would be separate server hosts and any incoming requests are distributed amongst them to balance load. Large scale web services are architected this way since single servers wouldn't be able to respond to the amount of traffice they'd receive. 

We don't have spare server racks kicking around to demonstrate this on but we can see what it looks like with one manager node at least.

**Step 1:** Create a directory called `hello_swarm` and copy the following into it as `hello_host.py`:

In [ ]:
import os
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return F"Hello from host {os.environ['HOSTNAME']}"


**Step 2:** Define a Dockerfile like those used for Flask applications already but with the `FLASK_APP` value set to `hello_host.py`.

**Step 3:**
Build the image tagging it as `hello-swarm`.

**Step 4:**
Initialize Swarm with with the command `docker swarm init`.

**Step 5:**
The command for join it gives you could be run on other hosts to add them as worker nodes, we could create a VM with Virtualbox or use Docker-machine to do this for us, but for now we can run a service with multiple instances on your single node.

We want to create a service, specifying our image as the one to use (or multiple images with Docker Compose) with other flags to specify name, number of replicas to use (ie. how many copies of the container to run), and port routing. Run the following:

```sh
docker service create --name hello_swarm --publish published=5000,target=5000 --replicas 2 hello_swarm
```

**Step 6:**
We want to now query our running application through the IP address mentioned in the `docker swarm join` command suggested when you ran `docker swarm init`. This is the external interface IP address Swarm is listening to for incoming requests. You can open a browser and navigate to that IP and port 5000 or use `curl` if you can't access the machine directly. What you expect to see is the server responding with 2 different host names, one for each replica.

**Step 7:** Inspect what services are running with `docker service ls`. Our `hello-swarm` service is running, we don't need it now so kill it with `docker service rm hello_swarm`.

So if we have an application (ie. a web server running in a container serving pages and other media to clients), how do they get data, communicate with each other if necessary, communicate with databases and synchronize their contents, and otherwise behave like a single server? One way is with volumes which can be created through Docker so that containers can access and share file data. Other running services can be set to serve multiple workers, ie. having a container running the database multiple instance of a web server access. 